In [1]:
import re
import pandas as pd
import numpy as np
import regex

### Python code to extract Date from chat file 

In [2]:
def startsWithDateAndTime(s):
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    #pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    pattern = '[0-9]{1}/[0-9]{2}/[0-9]{2}' # for New Group settings
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [3]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [4]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [5]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'WhatsApp_Chat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    
    #print(fp.readline())
    #fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(100)
            if len(messageBuffer) > 0:
                #print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

29957


In [6]:
df

,Date,Time,Author,Message
0,2021-05-11,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...
1,2021-05-11,8:33 PM,Celal PUCIT,Karain gey 👍
2,2021-05-11,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,2021-05-11,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,2021-05-11,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶
...,...,...,...,...
29952,2021-08-17,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...
29953,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr
29954,2021-08-17,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂
29955,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂


In [7]:
### Checking shape of dataset.
df.shape

(29957, 4)

In [8]:
### Checking basic information of dataset
#df.info()

In [9]:
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)

'''Don't drop, just take the rows where EPS is not NA'''

#df = df[~(df['Message'].str.len == 0)]
df = df.replace(r'^\s*$', np.nan , regex=True)
df = df.replace(r'<Media omitted>$', np.nan , regex=True)
df.shape

df = df[df['Message'].notna()]

In [10]:
### Checking no. of authors of group
df['Author'].nunique()

34

In [11]:
### Checking authors of group
df['Author'].unique()

array(['Ayesha PUCIT', 'Celal PUCIT', 'F.Bilal Uol', 'Umar Ghaffar PUCIT',
       'Amna Ali PUCIT', 'Ayesha Kiran PUCIT', 'Abdullah PUCIT',
       '+92 323 4446586', 'Khurram PUCIT', '+92 322 4155225',
       'Saad PUCIT', 'Adnan Ashraf PUCIT', 'Iqra Aslam PUCIT',
       'Shakir Jamil', 'Muflah PUCIT', 'Arshia CH PUCIT',
       'Usman CH PUCIT', '+92 304 5307578', '+92 300 4611646',
       'Muhammad Faraz PUCIT', '+92 306 2628299', 'Hafsa PUCIT',
       '+92 321 7222267', '+92 307 3900838', 'Aimen PUCIT',
       '+92 303 5018620', 'Nabeel Jutt', '+92 322 6678540',
       'Saad PUCIT MSV', '+92 335 0607000', '+92 335 5027377',
       '+92 306 4093886', 'Aneesa PUCIT', '+92 303 9326488'], dtype=object)

### Adding one more column of "Day" for better analysis, here we use datetime library which help us to do this task easily.

In [12]:
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)
### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
#df.head()
#df.tail()

### Counting number of letters in each message
df['Letter\'s'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Word\'s'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN, x)).str.len()
media = np.sum(df.Media_Count)

### Looking updated dataset
df

,Date,Day,Time,Author,Message,Letter's,Word's,Url_Count,Media_Count
0,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...,57,12,0,0
1,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Karain gey 👍,12,3,0,0
2,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta...",51,7,0,0
3,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,46,10,0,0
4,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶,46,11,0,0
...,...,...,...,...,...,...,...,...,...
29952,2021-08-17,Tuesday,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...,77,17,0,0
29953,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr,20,5,0,0
29954,2021-08-17,Tuesday,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂,27,6,0,0
29955,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂,3,1,0,0


In [13]:
### Extracting basic statistics from the dataset
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
links = np.sum(df.Url_Count)
print('Group Chatting Stats : ')
print('Total Number of Messages : {}'.format(total_messages))
print('Total Number of Media Messages : {}'.format(media_messages))
print('Total Number of Links : {}'.format(links))

Group Chatting Stats : 
Total Number of Messages : 27694
Total Number of Media Messages : 0
Total Number of Links : 0


In [14]:
#Extracting basic statistics of each user : 
l = df.Author.unique()
for i in range(len(l)):
  ### Filtering out messages of particular user
  req_df = df[df["Author"] == l[i]]
  ### req_df will contain messages of only one particular user
  print(f'--> Stats of {l[i]} <-- ')
  ### shape will print number of rows which indirectly means the number of messages
  print('Total Message Sent : ', req_df.shape[0])
  ### Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word\'s']))/req_df.shape[0]
  w_p_m = ("%.3f" % round(words_per_message, 2))  
  print('Average Words per Message : ', w_p_m)
  ### media conists of media messages
  media = sum(req_df["Media_Count"])
  print('Total Media Message Sent : ', media)
  ### links consist of total links
  links = sum(req_df["Url_Count"])   
  print('Total Links Sent : ', links)   
  print()
  print('----------------------------------------------------------n')

--> Stats of Ayesha PUCIT <-- 
Total Message Sent :  1619
Average Words per Message :  5.920
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Celal PUCIT <-- 
Total Message Sent :  617
Average Words per Message :  7.170
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of F.Bilal Uol <-- 
Total Message Sent :  3338
Average Words per Message :  9.190
Total Media Message Sent :  77
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Umar Ghaffar PUCIT <-- 
Total Message Sent :  1649
Average Words per Message :  7.010
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Amna Ali PUCIT <-- 
Total Message Sent :  230
Average Words per Message :  8.260
Total Media Message Sent :  0
Total Links Sent :  0

-------------------------

In [15]:
data = [df['Message']]
rd = pd.concat(data, axis=1)

type(rd)

pandas.core.frame.DataFrame

In [16]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27694 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  27694 non-null  object
dtypes: object(1)
memory usage: 432.7+ KB


In [17]:
### new Data Frame
#rd = df['Message']

### Remove all Emoji and other Symbols
rd = rd.replace(r'\W*$', '' , regex=True)

rd = rd.replace(r'^\s*$', np.nan , regex=True)

#rd = df[df['Message'].notna()]
rd = rd[rd['Message'].notna()]

In [18]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24457 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  24457 non-null  object
dtypes: object(1)
memory usage: 382.1+ KB


In [19]:
rd

,Message
0,Hmme na pagal ho chuki har teacher ki mintaien...
1,Karain gey
2,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,Or phir teachers k krny waly kam b me he kron
...,...
29951,😂😂😂😂 Bus Focus on your study you will get soon
29952,I don't need that type of aurat jo MSCS ki wja...
29953,😂😂😂 Fir to chutti kr
29954,Aho chutti hi ki hoi hai


In [20]:
rd = rd.sample(frac = 1)

In [21]:
len(rd)

mode = [''] * len(rd)

print(mode[:10])

rd['Mode'] = mode

### Rearranging the columns for better understanding
rd = rd[['Mode','Message']]

rd['Words'] = rd['Message'].apply(lambda s : len(s.split(' ')))

['', '', '', '', '', '', '', '', '', '']


In [22]:
rd

,Mode,Message,Words
23040,,Yesh,1
9032,,Headshot,1
16534,,Me dupehr nu lm tey v jna,7
3940,,Kia pta nikal aye wo phir kon zumedar ho ga,10
16809,,Me v apna u fone da number on kranda,9
...,...,...,...
7080,,@923497155520 bta bhai.... Aiisa kya hua k ank...,11
10170,,oo bai kis liye,4
6863,,Abhi btany ki zrorat bhi rehti hai k kon hai,10
15380,,Agr hum laundo ne bnana hta to pehly din hi bn...,12


In [23]:
#rd.to_csv('data.csv', index= False)  ## file size 20k+ ......

In [24]:
rd = rd[rd.Words != 1]
rd = rd[rd.Words != 2]

In [25]:
rd

,Mode,Message,Words
16534,,Me dupehr nu lm tey v jna,7
3940,,Kia pta nikal aye wo phir kon zumedar ho ga,10
16809,,Me v apna u fone da number on kranda,9
3744,,Hahah tuso km khrab e krna,6
19456,,Subha 8 bje uni ma mil tujhe quiz ko tiyari kr...,13
...,...,...,...
7080,,@923497155520 bta bhai.... Aiisa kya hua k ank...,11
10170,,oo bai kis liye,4
6863,,Abhi btany ki zrorat bhi rehti hai k kon hai,10
15380,,Agr hum laundo ne bnana hta to pehly din hi bn...,12


In [26]:
message = rd['Message']

In [27]:
type(message)

pandas.core.series.Series

In [28]:
for i in message:
    print (i)

Me dupehr nu lm tey v jna
Kia pta nikal aye wo phir kon zumedar ho ga
Me v apna u fone da number on kranda
Hahah tuso km khrab e krna
Subha 8 bje uni ma mil tujhe quiz ko tiyari kr du ga
Umed hai tujhe jawab mil gya hona
Ma smjh raha hu jo tu kehn chahta hai pr kehny se kia hota hai
Mei, Faraz, saad, umar etc
Mei tou subho say abh mobile pkra
Ruko me sharbat pilata hn apko roooh afza
Baat ice-cream wale ki o rae ap kaha nikal o gye o
Ak do bar kah k ab bndy ki hwa tight ho gai hoi h
Yrrr sb ignore kren.. group hi leave kr jaen
NLP k lecture ma to waqt jesy thum sa jata hai. Akhri adha ghnta mra 5 ghnty ma guzra tha
Bsss mere se lazmi milwaye ga.. baqi me sb puch lu gi😂😂 blke smjhane ki b puri Koshish krun gi
Mujhe bisti feel e ni hoti. Atni to mera warm up ha
Bhai tujh sy pahly vo mera love ha last 5 years sy
Ye acha diya
Mushkil kam hai yar wo
is ko m wese hi ignore krta hoan
Umeed to h
Acha acha to aisy kaho na
Shakir tu tw na e bol na
Yr marks hoty to rull jaty bai.. Free k marks ha

You deleted this message
yaad unko kia jataa Jinko bhula dia jataa...tje to bhulte e ni
Wo mera dost hai. Alag baat hai k chor gya hamain
meri mphil iT h
Mjy to RM lag rhe
Yarrr python tough ni h
Chalakiya ty o vi saady nal. Koi ni hisaab pura kr ly gy uni to open ho jy
Tere quiz ki tyari pe lanat
Mei b apnay upar walay msg main indirectly yahi keh chukii hu
Yay cpia ma kia tha na inonay
tujy yr bar bar batana parta pta tu h
Hamri class fellow banny pa dukh ho rha ha kya
Yar esy online ni hta proj jo bt hai
Ghalat jaga agya me... Oh sorry
100% project likha h yar
ALLAH maaf karey mujhay
Ni kisi zamany me... Me ne email ki thi uska reply tk na aya
Bht e khobsorat tasweer Hy ye apki😌😌alawww
proud of myself
Sir ni aaye abi
😂🤣😂🤣 Me smjh skti hun.. jb hr jga se heavy pr rhi hon to bnda idhr udhr chla hi jata
Link bhjo lecture ka
Han na dilo me ranjishein le kr bethe rho or mu pe meethe bn jao Aese Munafikat ni ati humein
Hahahah chalo sahi ha
Bacha gussa kr gya
Bai Me Faisalabad ka nai
🤣🤣🤣🤣

Abdullah ab status na lgana
Kuch na bolny sy to behtar tha
🤣🤣🤣Aisay kese yaqeen s keh dia
G main ny bhe kia tha sir ko msg
Shuru ho chuka hai
Saad suno zara....dost sath lga hua ya ahmmmm
Itny bary bary shayar logon ki mehfil m ni bolna chahiye
Yaar esy dakh to Mery mind ma kuch aur e a rha ha
@923496838548 btayn gi apko
Ye to hmara farz tha. Is ke shukrya kesa
Usky opinion py kisi ny argu nhi kiya or na he criticise kiya hai.. Uper ki chat parhien ge to apko pata chal jay ga k bat kiya the
Tera yar h celal
Na puch yar
🤣🤣 yeh acha ta. Pr koi tara bna hi ni
Yr kurrio hm b apna ek group bna Len?🤔🤔shugal mely Ka shugal Mela kaam Ka kaam
Me kehti hun soch len
Sahi hi gaya
😂😂😂 phr ap ne b roz ek video mangni he is se
In logon k liye 3 ghanty ki khamoshi honi chahiye
@923054524120 smjha yaar.. Smjh nai a rhi ch shb ko
Agar eid kal huee tou meri ACA walay sir k baray shughal main ke gaee baat sach ho jani k unho nay chaand raat ko assignment day k kehna eid ke nmaaz parkh k 10 bjay tak submit

Ghalti p ghalti krty ho bsh ap
Sahe hogya beeero
Majay Majay ka
But kbhi semester k darmiyan bhi to summer break ni hoti
Yr wesy ek bat kehna thi
Shukar krien bs group ko uthya hwa h
Ohhh bhai sb
Ap kia baar baar video dekh rhe hen
g bro Allhmadullilah
Bata rha ha ya pcuh rha ha
chalo saad khair jo h stay blessed
Celal e laga huva ha. To khud e samjh ja kuch b important ni ha
Isko 2 lgata ma ye sath ma hota to
Online ka suna ha
Yaha to thnd lg rahi hai
Nahe yar waqe ma...aksar CR's itna kabhe nahe krty
Vo parh rhi hn
Shaper double kr
Hn keep it up
Indirectly to 3no ka e msla hai
ja ja tur ja
itna time hi ni h
Meri ToC mein 50 they aur wo apni galti ki waja sey they. So jo hua theek hua. AOS mein nay apni galti ki waja sey drop kiya aur ab ACA parha. So, kuch bhi galat nhi hota. Banday k apnay blunders hotay
Fawad khan is asking for your location
Hmm ok bro
Jo kaam na guzray wo galat toh nhi hota
Puray lecture yahi issue raha
thanks for info media waly dhond rae apko
Nai 😂me saffr krna


Mei kia btaun
Ma khailta hu tennu koi msla
Tu nikal aj
Tum log itny senti kiun ho jaty jaldi e
Han ya b sae ha
Ab ho ri h
Seedhi tarh khien apko ap e fiqar ha .. As client
Mera mtlb tha female py crush ni hata mujhy
Jb Rockstar aye ga then mention kr dena  Me Dare complete kro Or Darwana bnd kr k chala jao ga
Yeee mje lg rahaa he k video pr zlalt ne dimagh pr asr kr dia he.. @923420431383 ? Nhi
Ab ye kis n kaha
saada liver hy apna
Ye to movie wala scene ho gya😂😂😂 Bechary sachy ashiq hony
Larkio se 19-20 ziada hain
Chk kr len koi muqabla hi nhi
han pta menu sb
Ooo bhai Apki class ka hi ho
Form submission kuch nhe phir
Saad to love e
Raat k pichly pehar
@923054524120 shb smjha lo me keh rha
Shoty shoty logo se panga ni lety @923484993573
You deleted this message
To phir hm aisy e dost hain
Happy birthday brother
Hahahahahhahah bol bol k sbko scheme bta de
Haha ye dukh to ap behter smjh skti hain
It means i have to go by myself
Lag to yhi Ra Hy....kal se sir continuously lgy hue sari eval

Haha fer awain suttti jan deya
Mtlb phir ma bisti feel na kru
Yar wo hints dy dy ga k kaha se aa skta kesy questions aa skty hain
Muflah result agaya h kia mia ka
Emoji galt lg gya
girls ki bat man sakhty hain
Aasma pareshan nh hn .. Updated folder send kr diya tha.. And he accepted
G g me poch Lun gi
Don't forget to mention k yw headshot apko lga hai
Axa khushi ki lehar dorny e lgi thi k ap logo na rok si
@923344635920 ap se zada pyari he na
Dr imran farid
Tu to class ma hota b ni aisy e bol ra h
You're 100% right. Aur ye kaam sirf Hira nay nhi kiya. Aur aur bhi kaafi khawateen nay kiya and I know them very well
🤣🤣🤣🤣Wo to hai
phir chup e reh
Ha kr do
Is time Abdullah ro raha he Chup kr jao saare
Ty thori bisti v feel kr ly ga
Ye kb sy ni khola
Aj raat group call hoti ha phr apni. Tera ya gila door kr dy gy
Ary abi tw mny b jana tha
Warna choka aram se ajana ta agr thora prh liya hta
Bs glti hgai
Hamid Ashhad khan kn hain ab
Tu abi ni so rha
lekin me aggressive ni hu
Kr ley bai sb ky k

oske extra pay krte honge
Aho ey tey hqiga
Phr abhi ap n @923054524120 ka andaz e bayan nai dekha
Itni bari duniya me choti choti batein hti rehti
Ni ni atna b ni hu
Jo ap dkh rhy
Tu chatha kya ha
Tu thora late aya a
Chal tjhy b bta dun gi Thora sabar kar
Chal bs kr dia
Bro drive py sy lni hti tw classroom sy b download kr skti the m
Me CMS pe dekh ra ta
Mazrat kr li he na pehle hi
Bado k samny bnda khul k enjoy ni kr pata iss liye celal ko drop kr do sb
ni yr leave q krna
Sir ka achar dalna h
Final 50 hai
Ye kely se bndy mar deta hau
Ab cover na kren.. me b kahun k ye video q mang rhe hen baar baar
M to ap k reaction ko kha h unko to kuch kha e ni h
Jo mrzi btao bs bata do aik dafa
Logic hi logic h... Baiii jitni khushi apko ho rhi thi... To phir ye hi banta tha
Ap pehle @923497155520 ki sehet dekhen phr ye baat dobara kahen🙊🙊  Acha Sorry
Aho ik cheez jab mujhe kuch bolta ha na to Mery masg pa reply Kia bcz ma ny group mute kia huva to asy pta ni chalta
bilal kya type kar rae itni der

Mjy majal h ek b shayr ki smhj ae ho
Han but hum log to again b gy thy to es lie pta ha
Hahahahahahha... Unko force na krien.. Wo kr b lien gy
Bush kr de
Or andhera b
Ab asa b ni ha. Try bhai ki eidi a gai ha vase
Saad wo gali yad kr jo tujhe abhi sunai thi ma na
Ni rehn deyo
Saad tu Bhai h osko
Agar kal krwani h to kr dena
Ruk zara sbr kr
Ak member km jana dfa kr
un k liye r boht hain tum apni fikar kro
Naam kuch suny suny se lag rahe hain skills k
Hayee. Thank you iqra. Ap ny mujue es qabil to samjha
First wala phd k liya hai or second wala hmary liya hai
Daily guzrta samny se
Phir message krty hain group m
Sure sure me Kar dun gi
Ab ap chup e rhain
Be calm yar
Yaar apne chahne walon ko yun tarpaya nai krty hty
@923008982661 @923054524120 @923497155520 @923330431497 @923081550903
Ab ya Fm wtsap kon c hoti ha
Ni m pehchan k liye bta ri hon
Ye built in hti h seekhny sy ni ati
Khany piny ka
M ny to baii kisi ko tang ni kia
Jo khud is me pehl krta
Sun k boht acha feel howa k chalo apko y

Wo kon hain ab
🤨🤨 mids or final ka mtlb koi scene nhi
Yr chup o jao saare paapio
M smjh gai
Tu ty jaan ha meri but ab ya na puchi kon sy vali
Bus ab time guzar gea hai
Abay kameeeno knsi philosophy prh rhe aj kal @923054524120 @923330431497
@923344635920 shb b isi m hn.. Or @923081550903 b
Han na kaisa lga
Itnay foolproof nizaam sey vaccine lag rahi hai. Ye kesay ho gya
Ya konsi bimari ha
Kis bat ka gussa h
Haha.... Aj tak ni mily hum log
Yehe cheez tje bhabi kahey ge jb 1 br tre sasti acting dekhe ge
aby dhakan شیر ni سیر
Yeshh bataon inko Yar
Me unhy hi keh ra yara
Smhjdar ho ap
Saady tri b kro
Block fr v chal jata 🤭 pr yahan aisa koi hai e  nai
Don yaha pe aik hi hai
Aj Sunday h
sir ki post k neche
Ab ap sirf mtlb keliye ayein gy group me
Saad to jaan e apni
pehly adnan ki ti bari ab meri agai
Bilal wo pic hai abi bhi mery pas
shashky of hostel loif
Itna zaleel kia tha usko tb bbi left na krta to aur kia krta
Ha yr bat kr lety hain ak dfa ye zrori hai call py bat kr lety hain
Acha a

Sb choren.. @923497155520 han baii chas a gyi k nhi
phr marna hi bnta
Ya for each
bhai e tu apne
Sab is laik ni thy... Lakin jo thy... Khair kya kahein... Janey den
Ye sunny se pehly ma mr q nahi gya
Hahahahha saad Love he tu.... Tu mind ni krta Wrna kuch log hote apna personal gussa group me a kr nikalte
khud e to bola tm dono ne
Cable nahi aye itne saal ab ja k ai warna sirf ptv or atv hi thy
Baaiii in k liye to meetha hi tha naa
ab m kya ss send kron
ja kam kr apna
Tu n install ki. Hui
Bai enu kini wari akhya e pr e free e ni hunda
Han zahri c bt ha.. Ab itne b bongi nh
tujy m ny qatal kr dena 1 tu tum mery bary ghalt information dete ho r mere khilaf krtyho
🤣🤣🤣 bhai mjhy ni chaye aisi khushi
Mjy b seekha dien
Ap ny saad ko involve kia tha
Haha inni jaldi hooron sy milny ki
Good ho gya
Mujhy to ab umeed b ni h
apka to hisaab lagate aj
Kab start hua
Sadiya sannu e
Chal enjoy kr
Mtlb ye kiya tha😂😂😂 Thandi chuss
Allah sbr de baqion ko esi toheen pr
hahahahhaha bht taiz o
Typing mistake

In [29]:
import re
reg = '[^\sA-Za-z]'
s = 'Sir bus hamri thori izat rakh rhy hn aur methi methi kr rhy hn'
s= '😂😂😂😂😂😂😂 error to me beech me se nikal lu ga oska fiker na kro'
rs = re.findall(reg,s)

In [30]:
len(rs)

7

In [31]:
def remove_emoji_msg(msg):
    import re
    reg_Exp = '[^\sA-Za-z]'
    len(msg)
    new_msg = []
    for i in msg:
        rs = re.findall(reg_Exp, i)
        if len(rs) == 0:
            new_msg.append(i)
    return new_msg

In [32]:
_list = remove_emoji_msg(message)

In [33]:
_list[:10]

['Me dupehr nu lm tey v jna',
 'Kia pta nikal aye wo phir kon zumedar ho ga',
 'Me v apna u fone da number on kranda',
 'Hahah tuso km khrab e krna',
 'Umed hai tujhe jawab mil gya hona',
 'Ma smjh raha hu jo tu kehn chahta hai pr kehny se kia hota hai',
 'Mei tou subho say abh mobile pkra',
 'Ruko me sharbat pilata hn apko roooh afza',
 'Ak do bar kah k ab bndy ki hwa tight ho gai hoi h',
 'Ye acha diya']

In [34]:
len(_list)

14282

In [41]:
print(type(rd))

for index, row in rd.iterrows():
    print (row["Message"], row["Words"])

<class 'pandas.core.frame.DataFrame'>
Me dupehr nu lm tey v jna 7
Kia pta nikal aye wo phir kon zumedar ho ga 10
Me v apna u fone da number on kranda 9
Hahah tuso km khrab e krna 6
Subha 8 bje uni ma mil tujhe quiz ko tiyari kr du ga 13
Umed hai tujhe jawab mil gya hona 7
Ma smjh raha hu jo tu kehn chahta hai pr kehny se kia hota hai 15
Mei, Faraz, saad, umar etc 5
Mei tou subho say abh mobile pkra 7
Ruko me sharbat pilata hn apko roooh afza 8
Baat ice-cream wale ki o rae ap kaha nikal o gye o 12
Ak do bar kah k ab bndy ki hwa tight ho gai hoi h 14
Yrrr sb ignore kren.. group hi leave kr jaen 9
NLP k lecture ma to waqt jesy thum sa jata hai. Akhri adha ghnta mra 5 ghnty ma guzra tha 20
Bsss mere se lazmi milwaye ga.. baqi me sb puch lu gi😂😂 blke smjhane ki b puri Koshish krun gi 20
Mujhe bisti feel e ni hoti. Atni to mera warm up ha 12
Bhai tujh sy pahly vo mera love ha last 5 years sy 12
Ye acha diya 3
Mushkil kam hai yar wo 5
is ko m wese hi ignore krta hoan 8
Umeed to h 3
Acha acha 

This message was deleted 4
Mean ap cr ni banna chati  na 8
tu tu soja 3
Han naw.. Iska ka b koi haal nikalti hn🤣🤣 6.27 10
Bs bs baithy rahien 4
its not my headche 4
Izat karwanay ka bara shoq h touba 7
Kis na kia ha ma na nahi dakha 8
Ab cover to na kren 5
Tra bilal ka mzaq hai 5
Ooo bai.... Bus kr de 5
Jo galiya ma ny dusry group ma awais ko di hn na Kash vo yaha tujhe b dy sakta 19
Sir ny sessional ma ajeeb chuss mari ha. Jin ki hum ny copy ki ha un k kam hn aur Jin ko copy karvai hn un k zyda hn 29
Apka just msg tag hwa hai .. Apko mention nh kiya gya.. Again kahun ge dill py na lien ap 20
Jan'ny ki khwahish bhi ni hmain 6
😂😂😂😂me b yhi kehny lgi thi 6
Saady tujy Facebook sy meri koe achi pic ni mili 10
Abdullah ko class ly rai hai Aasma 7
12 bajny walay hain aur eid ka koe pata nahe 10
Muflah inko na khilana 4
Ye tooo baaiii sahi fit ho raha.. @923204196291 @923081550903 smjh rhe ho naa?? Kitna meetha tha lehja 17
Phla msg he time sy ata .. Lgta h ab baki phir rok gy hain😶😶😶 06.18 1

Tra pta e na mje 5
Hahahha.. Nh yr Msgs receive hoty lkn send nh hoty 10
Hamny tu idar manani h 5
Kesi ashleel guftgu ho rhi hai 6
abi decison na lena 4
@923008982661 dekh le ab issy 5
challenge ja kam kr 4
Ma late ho gya 4
ab tu manana chaye sir ko wese 7
mere 4 saal batch mate r apki abi 1 saalhoa 10
Q ma koi abdullah hu 5
bikul sahi kaha ayesha ne 5
Jo baat buri lge keh do Phr Bacho wali smjo ya abu wali baat 14
Lakin ye hain kon kon 5
Okay thank you 3
🤣😂Pehle to kabhi nai btaya 5
@923106905056  cr yeh kya س hy 7
Ohh.. Half time m krwa skty ho ap 8
Baii ayesha kiran na itny din bad msg kia hansan to bnta tha na 14
Drny wali cheez ni hai wo ap ly ayen jo b foji ly k any hain 16
Umar tuj se ni hota to bs 1 min k loye mujhe bna admin 14
Keh kon raha hay 4
Yr itna mza ata sir k lecture main.... Prhai b ho jati aor shugl mela b lga rhta 18
But ma'am assignments dain gi 5
Ab ap ko to sb ata h ap na dro 10
Han na... Hmari eid ka dosra din bera gark to na hta 12
wo batien hi ti bas 5
Kya la

😆😆😆🤣🤣🤣 shukr he clear kr dia 6
🤣🤣🤣 twano har gal da pta 6
Nhi ae mzak ae 4
😂😂 ab aimen ka Mobile srf gire ga nhi.. 🙊 baqi smjhdar hen ap 14
Message delete krne ka faida 5
Ap aur mei 2 ho gaiy abh ap aur mei mil k baki 5 dhoondhtay hn 16
Sach me 1st semester me kuch nai prha 8
tu menu gal kad ly ody nal compare na kr 10
Bus bus nechy a jao ab. Gir na jana 9
Chal ja shabash dakh movie tera itna hissa hi kafi hai abdullah ko alvida kehny k liye 17
Hmary to ye scene hota tha time se ja k bhi end py e baithna hota tha 17
Chal gin k bata mery kitny masg hn hn kal k din ma 13
Apko to smhj aa gai naw 6
Kia batein? Konsi batein 4
Dekh bhai veer apni jagah or bryani apni jagah 9
mujy apni category m shamil na kro tum dono 9
Bai tu chal dfa ho ja mere kehne pe aya ni tu 12
Kb se preshan ora 4
😂😂 Itna follow krty hain mjy 6
Itni lmbi discussion hui thi is bt py 8
Mery mssg dono ignore mary ja rhy ho😒. Bye ma jata hu 12
Briyani khany aye gy 4
group ki privacy to on kro bnda reply to kr se 11
Tm hm 

Khubsurt word ki bisti kr rha ha tu 8
Meri 2, 3 jannay wali orhi hue hn PUCIT old campus say unho nay kaha sir murtaza ka course na rkhna 21
Ya ha ha 3
Ha yar issy yad dila 5
Abhiiii btaya mene ap koo k @923224155225 ko mention mt kren.. unhe nhi acha lgta 15
Yhi bt smj nai ai 5
Ha to aur kia tu jo hmain ghaseet raha hai 10
Han agr market pump ho jy bari koi heavy vali 10
Bilal soun rhy ho 4
Naraz ho gai to mnai ja skti hain agr kisi din video dakhty dakhty agli k sans e bnd ho gaye phir 22
Bilaly or shakir to tum log kya rakh rey subject 10
Allah kry ho jy ni to kam khrb ho ga mera 11
Na parho but apni zamidari pa. Hame kcuh na kehna age ho gya to 14
Bhai itni Choti mehfil main zaib nhe deta 8
InshaAllah maan jaen gy... Hoping for the best 8
Hahahhahah😂🤣😂🤣😂 ye lit tha 4
Ma kal machine ly ao ga. Sab weight e kr ly gy 12
WaAlikum Salam Khair Mubarak 4
Well ap ka hisab kitab b meri nend sy zyda important nai h 13
🤣🤣🤣Noice step.. Umeed krty hn hm ch shb smjh jayein gy 11
Emotional bird na

...

Hahahha wah wah wah 4
Chup kr ja. Fir tu boly ga mujhe q nikala 10
jb ni krna tu tu q batata h 8
To phir new group ma kis kis ko lana hai 10
Yaha koi asa banda ha jo Allama Iqbal open university sy B. Ed kr rha ho ya kr chuka ho 20
Ap log class ma ho 5
Jo aaty hain un m sy b zyada ka ni pta 11
Ha thek ay jany diya 5
@923420431383 yaar aik naaty kahn wese 6
Just because she has a diff opinion from u does not make her empty minded she shared her point of view just like everyone else does. Instead of talking to her and resolving the issues sensibly she has u guys are doing this kind of conversation atleast criticism ko b sunny ka hosla lao 54
Tujhy challenge ho gya bilal 5
Iss se pehly k unar cahal jaya group chor k jldi share kr do 14
Mujhy yaad tha tu or khurram... Or tum dono ajao lahore kr len gy peelan 15
Aimen kya likhy ja rhi ho kh sy 8
Exactly my point 3
Ap ka hi shagird a gya tha m kya kahon 10
Dekh kr del kr do gy na 7
Usne to sab ko zero de deney 7
Nai nai laraiii nai krni 5
Ha

Saraikio wali pasand.... Mtlb smj ni aya mujyy 8
😅 Good question 3
🙊🙊🙊🙊🙊🙊 @923054524120 count down shuru kren 6
Teri himmat kesy hoi pubg ko pubj likhny ki 9
Yr to eid me bhi prh raha 7
Seedha seedha janwar hi keh den.. ye shair kehne ki kia zrurt thi 13
sahi kaha lekin final ki ratio 50 me se nikali thi...as u know paper 45 ka tha 17
😂😂 ye b sahi he 5
Saad ap b seekh jaien .. Kb bary hn gy 10
Ma b ni chorta bai. B 6
😂 to ehtyat kr na chorey 6
*Halo. Aiwari won. Haooo are uo gaiyez... Vaier are uo beeze na o a daiz 15
Raat k pichly pehar insan sahi gehrai ma hota hai na sahi ander tk munfiqat seekh skta hai 18
Lkn ye transcript py * wala kam khrb ha 9
Apki bt thek hai sir ny valid bt b ni suni hmari 12
😂🤣😂🤣😂🤣 Shakir shb aj kal khud smjh rhe hn 9
Eid Mubarak to all 4
Hona b chahye.. swag ata he 6
Mera BS mein 3.04 hai 😂😂😂😂😂🙉🙉 mein toh proud btata hu 11
Nhi nhi nhii.. ap ki is emoji se mje lg raha ap kahin or ja rhe hen.. ruk jaen yhin 20
Ek b chajj ka nh ha 6
Acha fer thik ho gya 5
Huka

😂😂😂😂😂😂 Hm sunty hain.... Magar sirf so shair... Jo soty howe ko jaga de 14
Osme bs kuch log hain saare boys ni hain 9
You deleted this message 4
Ma ny to suna tha k English cricket board sy b baat kr rhy hn.k hamry match host karva dy England ma 22
munh acha hy na is lye bat achi ni hoti 10
Ahki kro. Pr Mairey ko ni chaye RM da swag se kren gy sab ka swagat 16
Apna officials sticker b send krti na 7
Poch kon raha hai 4
Isko south indian bta 4
Nh nh.. Bht kuch seekhny ko mill rha ha .. I am happy 13
kon kon sy offer hoe hain 6
Wo choro.. Ap b dedicate kro bs 7
Paisa = Power (Never forget 5
Ye fazool system h iska kuch pata nai chalta 9
Me gia kb tha 4
Mokay pe choka 3
Abdullah ka saans ruk jana 5
Wsy apas ki bt hai NLP ki slides mry pas b aati hain means sbko e jati hain. Jinka subject nahi b hai unko b jati hai sir ki mails 30
Pakistanio ka bhi yahi haal hai 6
Ap gy nh abhe tk 5
😂😂btw parhaty kya hen ye 5
Na hum Muslman na Pakistani 5
Shoreya kia ota e 4
May be sir verify Kar k increas

Han ye to h kafi zlalt hui thi hmari b 10
Bilkul you are right 4
Vndy ka hosl e aur hota 6
ye mazay ka hay... buddy 5
Apka pehla experence hai na is liye adat ho jae gi 11
مدتوں بعد طبیعت ہے شگفتہ خاطر آج یوں ہی دسویں جماعت کی ثناء یاد آئ 15
😑😑😑😑 I can't say anything now 6
Mre pas gawah b tha 5
Keh kon raha hai 4
Ni yar dil se bura lgta hai jb tu jata hai group se lakin ab hm tujh se mzaq bhi na krain kia 23
Banta hai yr 3
Ye toh ap mere pe boht bara ehsaan kr rahay 10
Eid Mubarak howe biro .. @923081550903 6
Ap Muflah sy kya expect kr rhy hn 8
ni bai ap asa kr sakhti 6
Baii jb marks ki bat aa jaye phir to bazi lgani prti hai na 14
Responsibilty to hti h apki bt thek hai but there are many other factors jinki wjah sy insan ni b ly pata koi emergency hgai internet issue ya or cheexain 30
Jo cheez sir ny mangi e ni us ki base py kesy kat skty hain marks 16
Bus phr sae ha. Khushi hu sun k 8
HAha. Enu sitarey nazar awn gy 6
Hahaha me ni darda 4
To tu haar gya nikal ja yaha se ab 9
This mes

Ya Pooja kon ha class ma 6
Raat ko kr ly gy. 10 mint ka to kam ha 11
Han dakh k dislike kr dia 6
Bai ye hmari hi mairas hai 6
Bai aj ma serious hu bara. Ni sab pkr k hame zalel kar rhy hn. Ni aur jaga kam hn kya zalel hony vali 24
tum log apas m khush hoty raho 7
is age me humein in cheezo ka pta e ni ota tha....or aj kal 14 saal k bache relationship me hain 21
Yr ap sorry na kao 5
Q k aik student ziada se ziada or kia chah sakta h??? zehni sakoon. Agar wo hi management barbad kr de gi... to phir ap positive cheezain bhi bhool jaty ho 31
Yr Aasma be cool 4
Han g ya to sae kaha 8/1/21, 3:12 PM - F.Bilal Uol: Quiz 😭😭😭😭😭😭 8/1/21, 3:12 PM - F.Bilal Uol: Sir ko bolo next class ma kr ly 8/1/21, 3:18 PM - Shakir Jamil: Koi to roko sir ko 8/1/21, 3:19 PM - Abdullah PUCIT: abi parhany do abi parhaya kya h 8/1/21, 3:21 PM - Arshia CH PUCIT: Koi to roko sir ko(2) 8/1/21, 3:22 PM - +92 306 2628299: Koi to roko sir ko(3) 8/1/21, 3:22 PM - Muflah PUCIT: Congratulations.... Like I said.... Ye ain tim